In [1]:
import torch
from tqdm import tqdm
import random
import math
device = "cuda:0"
random.seed(42)

In [3]:
def retrieval(features_A, features_B, k_way=4):
    """
    由A检索B
    """
    features_A = features_A.to(device)
    features_B = features_B.to(device)

    assert features_A.shape[0] == features_B.shape[0], "features_A 与 features_B 长度不相等"

    logit_scale = math.log(1 / 0.07)
    batch_size = 20
    N = features_A.shape[0]

    all_classes = set(range(0, N))
    correct = 0
    for i in tqdm(range(0, N, batch_size), desc="retrieving..."):
        for label in range(i, i + batch_size):
            possible_classes = list(all_classes - {label}) # 199个其它可能的类
            selected_classes = random.sample(possible_classes, k_way-1) + [label]
            selected_features = features_B[selected_classes]

            logits = logit_scale * features_A[label] @ selected_features.T
            predicted_label = selected_classes[torch.argmax(logits).item()]

            if predicted_label == label:
                correct += 1
    
    accuracy = correct / N
    return accuracy

### EEG检索图像

In [4]:
# eeg_features = torch.load("/home/tom/fsas/eeg_data/features/old_features/ATM_S_eeg_features_sub-08_test.pt", weights_only=True)
image_features = torch.load("/home/tom/fsas/eeg_data/features/ViT-H-14_features_test.pt", weights_only=True)['img_features']

# 循环处理所有sub
for i in range(1, 11):  # 生成sub-01到sub-10
    sub_id = f"sub-{i:02d}"  # 保证两位数格式
    
    # 动态生成EEG特征路径
    eeg_path = f"/home/tom/fsas/eeg_data/eeg4image/{sub_id}/ATM_S_eeg_features_{sub_id}_test.pt"
    eeg_features = torch.load(eeg_path, weights_only=True)
        
    # 执行检索并输出结果
    result = retrieval(eeg_features, image_features)
    print(f"{sub_id} 检索结果:", result)

retrieving...: 100%|██████████| 10/10 [00:00<00:00, 59.37it/s]


sub-01 检索结果: 0.84


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 232.65it/s]


sub-02 检索结果: 0.83


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 192.04it/s]


...

sub-08 检索结果: 0.895


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 306.81it/s]


sub-09 检索结果: 0.445


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 216.86it/s]

sub-10 检索结果: 0.455


### EEG检索文本

In [6]:
# eeg_features = torch.load("/home/tom/fsas/eeg_data/features/old_features/ATM_S_eeg_features_sub-08_test.pt", weights_only=True)
# text_features = torch.load("/home/tom/fsas/eeg_data/features/ViT-H-14_features_test.pt", weights_only=True)['text_features']
text_features = torch.load("/home/tom/fsas/eeg_data/features/ATMS_ViT-H-14_text_features_test.pt", weights_only=True)

# 循环处理所有sub
for i in range(1, 11):  # 生成sub-01到sub-10
    sub_id = f"sub-{i:02d}"  # 保证两位数格式
    
    # 动态生成EEG特征路径
    eeg_path = f"/home/tom/fsas/eeg_data/eeg4text/{sub_id}/ATM_S_eeg_features_{sub_id}_test.pt"
    eeg_features = torch.load(eeg_path, weights_only=True)
        
    # 执行检索并输出结果
    result = retrieval(eeg_features, text_features)
    print(f"{sub_id} 检索结果:", result)

retrieving...: 100%|██████████| 10/10 [00:00<00:00, 346.78it/s]


sub-01 检索结果: 0.685


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 347.90it/s]


sub-02 检索结果: 0.695


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 346.98it/s]


...

sub-08 检索结果: 0.805


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 304.79it/s]


sub-09 检索结果: 0.435


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 351.03it/s]

sub-10 检索结果: 0.435


### 图像检索EEG

In [7]:
# eeg_features = torch.load("/home/tom/fsas/eeg_data/features/old_features/ATM_S_eeg_features_sub-08_test.pt", weights_only=True)
image_features = torch.load("/home/tom/fsas/eeg_data/features/ViT-H-14_features_test.pt", weights_only=True)['img_features']

# 循环处理所有sub
for i in range(1, 11):  # 生成sub-01到sub-10
    sub_id = f"sub-{i:02d}"  # 保证两位数格式
    
    # 动态生成EEG特征路径
    eeg_path = f"/home/tom/fsas/eeg_data/eeg4image/{sub_id}/ATM_S_eeg_features_{sub_id}_test.pt"
    eeg_features = torch.load(eeg_path, weights_only=True)
        
    # 执行检索并输出结果
    result = retrieval(image_features, eeg_features)
    print(f"{sub_id} 检索结果:", result)

retrieving...: 100%|██████████| 10/10 [00:00<00:00, 347.20it/s]


sub-01 检索结果: 0.86


retrieving...:   0%|          | 0/10 [00:00<?, ?it/s]

retrieving...: 100%|██████████| 10/10 [00:00<00:00, 351.77it/s]


sub-02 检索结果: 0.855


...

sub-08 检索结果: 0.915


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 226.85it/s]


sub-09 检索结果: 0.55


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 344.07it/s]

sub-10 检索结果: 0.525


### 文本检索EEG

In [8]:
# eeg_features = torch.load("/home/tom/fsas/eeg_data/features/old_features/ATM_S_eeg_features_sub-08_test.pt", weights_only=True)
# text_features = torch.load("/home/tom/fsas/eeg_data/features/ViT-H-14_features_test.pt", weights_only=True)['text_features']
text_features = torch.load("/home/tom/fsas/eeg_data/features/ATMS_ViT-H-14_text_features_test.pt", weights_only=True)

# 循环处理所有sub
for i in range(1, 11):  # 生成sub-01到sub-10
    sub_id = f"sub-{i:02d}"  # 保证两位数格式
    
    # 动态生成EEG特征路径
    eeg_path = f"/home/tom/fsas/eeg_data/eeg4text/{sub_id}/ATM_S_eeg_features_{sub_id}_test.pt"
    eeg_features = torch.load(eeg_path, weights_only=True)
        
    # 执行检索并输出结果
    result = retrieval(text_features, eeg_features)
    print(f"{sub_id} 检索结果:", result)

retrieving...: 100%|██████████| 10/10 [00:00<00:00, 236.77it/s]


sub-01 检索结果: 0.73


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 258.66it/s]


sub-02 检索结果: 0.71


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 233.24it/s]


...

sub-08 检索结果: 0.805


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 222.89it/s]


sub-09 检索结果: 0.43


retrieving...: 100%|██████████| 10/10 [00:00<00:00, 255.51it/s]

sub-10 检索结果: 0.49
